In [19]:
import numpy as np
import pandas as pd
import os
import plotly as py
from pulp import * 
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [20]:
files = ['C:/Users/denpo/Desktop/ДроныДипломУсловие/busy_day.in']
file_name = files[0]
with open(file_name) as file:
    lines = file.read().splitlines()
line_list = [[int(n) for  n in ll.split()] for ll in lines]

In [21]:
distance = lambda x1, y1, x2, y2:np.sqrt(np.abs(x1-x2)**2 + np.abs(y1-y2)**2)

In [22]:
R = line_list[0][0]
C = line_list[0][1]
n_drones = line_list[0][2]
T = line_list[0][3]
payload = line_list[0][4]

n_products = line_list[1][0]
product_weight = line_list[2]

n_warehouses = line_list[3][0]

warehouses = []
wh_stocks = []

for i in np.arange(4, 2*(n_warehouses) + 4, 2):
    warehouses.append( (line_list[i][0], line_list[i][1]) )
    wh_stocks.append( line_list[i+1] )
    end_wh = i+2

In [23]:
order_startline = 2*(n_warehouses+1)+2 
n_order = line_list[end_wh][0]

order_delivery = []
order_n_items = []
order_quantity_items = []

end_order = 3*(n_order) + (order_startline+1)

for i in np.arange(order_startline+1, 3*(n_order) + (order_startline+1), 3):  
    order_delivery.append(line_list[i])
    order_n_items.append(line_list[i+1][0])
    order_quantity_items.append(line_list[i+2])
    end_o = i+3

In [24]:
d_wh = {
    "warehouse" : [i for i in range(0, n_warehouses)],
    "x" : [i[0] for i in warehouses],
    "y" : [i[1] for i in warehouses],
}

for i in range(n_products):
    p_id = 'prod_'+str(i)
    d_wh[p_id] = []

for j in range(n_warehouses):
    for i in range(n_products):
        p_id = 'prod_'+str(i)
        d_wh[p_id].append(wh_stocks[j][i])

df_wh = pd.DataFrame(d_wh)

df_wh.head()

,warehouse,x,y,prod_0,prod_1,prod_2,prod_3,prod_4,prod_5,prod_6,...,prod_390,prod_391,prod_392,prod_393,prod_394,prod_395,prod_396,prod_397,prod_398,prod_399
0,0,113,179,0,0,5,1,0,0,0,...,3,2,0,2,0,0,11,6,7,0
1,1,234,599,0,0,3,1,1,0,0,...,7,9,0,1,0,0,5,7,11,0
2,2,195,89,0,0,1,3,0,0,0,...,3,0,0,1,0,0,5,5,5,0
3,3,215,207,1,0,4,0,1,0,15,...,3,4,0,3,0,0,6,5,8,0
4,4,220,326,1,0,6,1,2,0,0,...,3,0,0,3,25,0,2,2,6,0


In [25]:
d_order = {
    "order" :        [i for i in range(0, n_order)],
    "items" :        order_n_items,
    "total_weight" : [0]*n_order,
    "x":             [order[0] for order in order_delivery],
    "y" :            [order[1] for order in order_delivery],
}

for i in range(n_products):
    p_id = 'prod_'+str(i)
    d_order[p_id] = [0]*n_order

for j in range(0, n_warehouses):
    d_order["wh_"+str(j)] = [0]*n_order
    d_order["dist_"+str(j)] = [0]*n_order
    
for k in range(n_order): 
    w = 0
    for i in order_quantity_items[k]:
        p_id = 'prod_'+str(i)
        d_order[p_id][k] += 1
        w += product_weight[i]
           
        
        for j in range(0, n_warehouses):
            if df_wh[p_id][j] > 0:
                d_order["wh_"+str(j)][k] += 1
            else:
                d_order["wh_"+str(j)][k] += 0
            
    
        for j in range(0, n_warehouses):
            d_order["dist_"+str(j)][k] = distance(d_order['x'][k], d_order['y'][k],
                                                 df_wh['x'][j], df_wh['y'][j])
            
    
    d_order["total_weight"][k] += w
    

df_order = pd.DataFrame(d_order)
df_order['nearest_wh'] = df_order[['dist_'+str(i) for i in range(0, n_warehouses)]].T.idxmin().apply(lambda x: x.split('_')[1])

df_order.head()

,order,items,total_weight,x,y,prod_0,prod_1,prod_2,prod_3,prod_4,...,dist_5,wh_6,dist_6,wh_7,dist_7,wh_8,dist_8,wh_9,dist_9,nearest_wh
0,0,8,419,340,371,0,0,0,0,0,...,238.008403,3,269.135654,4,256.719691,3,346.301891,4,67.475922,9
1,1,1,45,163,320,0,0,0,0,0,...,128.413395,1,131.711807,1,191.376592,1,328.701993,1,169.011834,4
2,2,2,88,201,331,0,0,0,0,0,...,139.301831,2,153.117602,2,192.898937,2,323.891957,2,132.966161,4
3,3,10,702,257,290,0,0,0,0,0,...,122.613213,9,222.503933,8,152.777616,8,269.267525,8,138.884844,4
4,4,8,508,115,512,0,0,0,0,0,...,325.960120,7,102.156742,7,388.784002,5,523.661150,7,202.595656,6


In [26]:
n = 1150
order_quantity_items[n].sort()

In [27]:
# продукт k в складе j
stock = [[0]*n_products]*n_warehouses
for wh in range(n_warehouses):
    for prod in range(n_products):
        stock[wh][prod] = wh_stocks[wh][prod]
stock = np.array(stock)

# продукт k в заказе i 
demand = np.array([[0]*n_order]*n_products)
for order in range(n_order):
    for prod in order_quantity_items[order]:
        demand[prod][order] += 1
demand = np.array(demand)

# продукт k в складе j для заказа i
x = [[[LpVariable("x_"+str(k)+"_"+str(i)+"_"+str(j), 0,  min(1, demand[k][i]), LpInteger) 
       for j in range(n_warehouses)] 
      for i in range(n_order) ] 
     for k in range(n_products) ] 

# расстояние между складом i и заказом j
cols0 = ['dist_'+str(j) for j in range(n_warehouses)]
dist = np.array(df_order[cols0]).astype('float32')

In [28]:
stock = [[0]*n_products]*n_warehouses
for wh in range(n_warehouses):
    for prod in range(n_products):
        stock[wh][prod] += wh_stocks[wh][prod]

stock = np.array(stock)

In [29]:
# Проблема
prob = LpProblem("assign_product_to_order", LpMinimize)

prob += lpSum(dist[i][j] * x[k][i][j] for k in range(n_products) for i in range(n_order) for j in range(n_warehouses)), "obj"

# Ограничения            
for k in range(n_products): 
    prob += lpSum(x[k][i][j] for i in range(n_order) for j in range(n_warehouses) ) <= lpSum(stock[j][k] for j in range(n_warehouses)), "Stock cst"+str(k)

for i in range(n_order): 
    for k in range(n_products):
        prob += lpSum( x[k][i][j] for j in range(n_warehouses) ) == demand[k][i], "demand cst"+str(k)+"_"+str(i)



In [30]:
%%time

prob.solve()

print("Status:", LpStatus[prob.status])

# значение цели
print("objective=", value(prob.objective))

Status: Optimal
objective= 536097.366312027
Wall time: 2min 33s


In [31]:
%%time

z = np.array([[0]*n_warehouses]*n_order).astype('int32')
df_assign = pd.DataFrame(columns = ['warehouse', 'order', 'product_id', 'quantity', 'total_weight'])

for k in range(n_products):
    for i in range(n_order):
        for j in range(n_warehouses):
            if x[k][i][j].varValue > 0:
                z[i][j] = 1
                
                df_assign = df_assign.append({
                    'warehouse':    j,
                    'order':        i,
                    'product_id':   k,
                    'quantity' :    int(x[k][i][j].varValue), 
                    'total_weight': int(x[k][i][j].varValue) * product_weight[k]
                }, ignore_index=True)
                          
print(df_assign.shape)

order = 10
tmp = df_assign.sort_values('warehouse').query('order == '+str(order))
prods = tmp.product_id

(9368, 5)
Wall time: 7.68 s


In [32]:
df_assign.sort_values('warehouse').query('quantity > 2').head()

,warehouse,order,product_id,quantity,total_weight


In [33]:
%%time

for i in range(n_order):
    if sum(z[i]) > 1:
        print("more than 1 wh ",i, ", sum =  ", sum(z[i]))
    if sum(z[i]) == 0:
        print("0 wh", i)
     
    
for j in range(n_warehouses):
    df_order['assign_'+str(j)] = z.T[j]

more than 1 wh  3 , sum =   2
more than 1 wh  20 , sum =   2
more than 1 wh  45 , sum =   2
more than 1 wh  52 , sum =   2
more than 1 wh  54 , sum =   2
more than 1 wh  56 , sum =   2
more than 1 wh  88 , sum =   2
more than 1 wh  96 , sum =   2
more than 1 wh  98 , sum =   2
more than 1 wh  101 , sum =   2
more than 1 wh  103 , sum =   2
more than 1 wh  108 , sum =   2
more than 1 wh  111 , sum =   2
more than 1 wh  112 , sum =   2
more than 1 wh  116 , sum =   2
more than 1 wh  122 , sum =   2
more than 1 wh  125 , sum =   2
more than 1 wh  135 , sum =   2
more than 1 wh  163 , sum =   2
more than 1 wh  199 , sum =   2
more than 1 wh  200 , sum =   2
more than 1 wh  204 , sum =   2
more than 1 wh  210 , sum =   2
more than 1 wh  211 , sum =   2
more than 1 wh  216 , sum =   2
more than 1 wh  221 , sum =   2
more than 1 wh  229 , sum =   2
more than 1 wh  234 , sum =   2
more than 1 wh  249 , sum =   2
more than 1 wh  252 , sum =   2
more than 1 wh  282 , sum =   2
more than 1 wh  28

In [34]:
df_order[['order', 'items']+['assign_'+str(j) for j in range(n_warehouses) ]].head()

,order,items,assign_0,assign_1,assign_2,assign_3,assign_4,assign_5,assign_6,assign_7,assign_8,assign_9
0,0,8,0,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,1,0,0,0,0,0
2,2,2,0,0,0,0,1,0,0,0,0,0
3,3,10,0,0,0,1,1,0,0,0,0,0
4,4,8,0,0,0,0,0,0,1,0,0,0


In [35]:
fig = make_subplots(rows=1, cols=1)

ref_color = ['red','blue',  'brown', 'purple', 'orange', 'cyan', 'black', 'yellow', 'pink', 'green']

for i in range(n_warehouses):
    tmp = df_wh.iloc[i]
    color = ref_color[i]
    fig.add_trace(go.Scatter(x=[tmp['x']], y=[tmp['y']],
                             name='warehouse '+str(i),
                             marker_size=15,
                             marker_color=color,
                             mode='markers', 
                            text='warehouse '+str(i)
                            ), row=1, col=1)

    tmp_ord = df_order.query("assign_"+str(i)+" >= 1.0")
    print(tmp_ord.shape[0], " orders for warehouse "+str(i), ' colorized in ', color)
    
    fig.add_trace(go.Scatter(x=tmp_ord['x'], y=tmp_ord['y'],
                            name='orders for warehouse '+str(i),
                            marker_size=3,
                            marker_color=color,
                            mode='markers'), row=1, col=1)

fig.show()

133  orders for warehouse 0  colorized in  red
25  orders for warehouse 1  colorized in  blue
56  orders for warehouse 2  colorized in  brown
295  orders for warehouse 3  colorized in  purple
356  orders for warehouse 4  colorized in  orange
128  orders for warehouse 5  colorized in  cyan
173  orders for warehouse 6  colorized in  black
83  orders for warehouse 7  colorized in  yellow
10  orders for warehouse 8  colorized in  pink
127  orders for warehouse 9  colorized in  green


In [36]:
df_assign.sample(5, random_state=89)

,warehouse,order,product_id,quantity,total_weight
2368,0,189,106,1,55
687,3,64,35,1,36
3910,9,183,163,1,45
232,9,289,15,1,93
512,7,987,27,1,74


In [37]:
warehouse_order = []
wh_order_dist = []

for j in range(n_warehouses):
    tmp = df_assign.query("warehouse == "+str(j))
    warehouse_order.append(tmp)
    print('Warehouse ', j,' : ', tmp.shape[0], 'products to deliver')
    
    #distances
    wh_order_dist.append([0] + df_order['dist_'+str(j)].tolist())

Warehouse  0  :  1005 products to deliver
Warehouse  1  :  137 products to deliver
Warehouse  2  :  386 products to deliver
Warehouse  3  :  2001 products to deliver
Warehouse  4  :  2467 products to deliver
Warehouse  5  :  691 products to deliver
Warehouse  6  :  1114 products to deliver
Warehouse  7  :  547 products to deliver
Warehouse  8  :  77 products to deliver
Warehouse  9  :  943 products to deliver


In [38]:
wh = 1
n_drone_per_wh = n_drones//10

df = df_assign.query("warehouse == "+str(wh)).sort_values('order')

order_list = list(df.order)

df.head()

,warehouse,order,product_id,quantity,total_weight
7251,1,98,307,1,62
15,1,98,2,1,84
5473,1,98,231,1,40
1321,1,98,63,1,11
5612,1,98,238,1,67


In [39]:
demands = df['total_weight'].tolist()+ [0]
products = df['product_id'].tolist()+[0]
len(demands)

X = np.array(d_order['x'])   
Y = np.array(d_order['y'])
         
ref_dist = np.array([[ distance(X[i], Y[i], X[j], Y[j]) for j in range(n_order)] for i in range(n_order)])

tmp = pd.DataFrame(np.array([[ref_dist[i][j] for j in order_list] for i in order_list]))

tmp[tmp.shape[0]] = pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0])
tmp = tmp.append(pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0]),  ignore_index=True)
distance_ = np.array(tmp)

In [40]:
pd.DataFrame(distance_).tail(5)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
133,25.079872,25.079872,25.079872,25.079872,25.079872,91.082380,18.384776,18.384776,18.384776,18.384776,...,114.127122,62.169124,62.169124,40.012498,0.000000,0.000000,0.000000,0.000000,0.000000,84.005952
134,25.079872,25.079872,25.079872,25.079872,25.079872,91.082380,18.384776,18.384776,18.384776,18.384776,...,114.127122,62.169124,62.169124,40.012498,0.000000,0.000000,0.000000,0.000000,0.000000,84.005952
135,25.079872,25.079872,25.079872,25.079872,25.079872,91.082380,18.384776,18.384776,18.384776,18.384776,...,114.127122,62.169124,62.169124,40.012498,0.000000,0.000000,0.000000,0.000000,0.000000,84.005952
136,25.079872,25.079872,25.079872,25.079872,25.079872,91.082380,18.384776,18.384776,18.384776,18.384776,...,114.127122,62.169124,62.169124,40.012498,0.000000,0.000000,0.000000,0.000000,0.000000,84.005952
137,89.844310,89.844310,89.844310,89.844310,89.844310,143.405021,72.006944,72.006944,72.006944,72.006944,...,109.444050,95.015788,95.015788,124.016128,84.005952,84.005952,84.005952,84.005952,84.005952,0.000000


In [41]:
turns = 30
n_drone_per_wh = 10 * turns
turns = [30, 30, 30, 100, 100, 100, 40, 40, 30]

In [42]:
def create_data_model(wh=0):
    """Stores the data for the problem."""
    
    n_drone_per_wh = n_drones//10
    df = df_assign.query("warehouse == "+str(wh)).sort_values('order')
    order_list = list(df.order)
   
    demands = df['total_weight'].tolist()+[0]
    products = df['product_id'].tolist()+[0]
    len(demands )

    # define distances 
    X = np.array(d_order['x'])   
    Y = np.array(d_order['y'])

    ref_dist = np.array([[ distance(X[i], Y[i], X[j], Y[j]) for j in range(n_order)] for i in range(n_order)])
    tmp = pd.DataFrame(np.array([[ref_dist[i][j] for j in order_list] for i in order_list]))

    tmp[tmp.shape[0]] = pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0])
    tmp = tmp.append(pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0]),  ignore_index=True)
    distance_ = np.array(tmp)

    turns = [30, 30, 30, 400, 400, 100, 100, 100, 100,100]
    turns = turns[wh]
    n_drone_per_wh = 10 * turns
    
    data = {}
    data['distance_matrix'] = distance_
    data['demands'] = demands
    data['vehicle_capacities'] = [payload]*n_drone_per_wh
    data['num_vehicles'] = n_drone_per_wh
    data['depot'] = tmp.shape[0]-1
    
    data['products'] = products
    data['order_list'] = order_list+[0]
    
    return data

In [43]:
%%time 
a = create_data_model(3)

Wall time: 11.4 s


In [44]:
def print_solution(data, manager, routing, solution):
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
   
    total_distance = 0
    total_load = 0
    count = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        
        if route_load > 0:
            count += 1
        
        total_distance += route_distance
        total_load += route_load
    print('Total needed rounds : {} '.format(count//10))
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [45]:
def main(wh=0, print_solution=False):
    
    data = create_data_model(wh)

    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    
    
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,
        data['vehicle_capacities'],
        True,
        'Capacity')

    penalty = 15000
    for node in range(1, len(data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT)
    search_parameters.log_search = True
    search_parameters.time_limit.FromSeconds(1200)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    print("Solver status: ", routing.status())
    if print_solution:
        if solution:
            print_solution(data, manager, routing, solution)
    
    return (data, manager, routing, solution)

In [46]:
def main_df(wh=0):
    (data, manager, routing, solution) = main(wh, False)
    
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))


    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))

    total_distance = 0
    total_load = 0
    count = 0

    df_route = pd.DataFrame(columns=['warehouse', 'route', 'order', 'product', 'load'])

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            prod = data['products'][node_index]


            tmp_data = {'warehouse': wh, 
                        'route': vehicle_id, 
                        'product': prod, 
                        'order': data['order_list'][node_index], 
                        'load': route_load }
            df_route = df_route.append(tmp_data, 
                                       ignore_index=True)

            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        if route_load > 0:
            count += 1
        total_distance += route_distance
        total_load += route_load
        
    print('Total needed rounds : {} '.format(count//10))
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    return df_route

In [47]:
%%time 

warehouses = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]

tmp_list = []
df_route = pd.DataFrame()
for wh in warehouses:
    print("\n\nWarehouse : ", wh)
    tmp = main_df(wh)
    tmp_list.append(tmp)



Warehouse :  0
Solver status:  1
Total needed rounds : 30 
Total distance of all routes: 43120m
Total load of all routes: 58213


Warehouse :  1
Solver status:  1
Total needed rounds : 4 
Total distance of all routes: 8367m
Total load of all routes: 9035


Warehouse :  2
Solver status:  1
Total needed rounds : 12 
Total distance of all routes: 11255m
Total load of all routes: 23969


Warehouse :  3
Solver status:  1
Total needed rounds : 68 
Total distance of all routes: 70358m
Total load of all routes: 132592


Warehouse :  4
Solver status:  1
Total needed rounds : 84 
Total distance of all routes: 97931m
Total load of all routes: 161801


Warehouse :  5
Solver status:  1
Total needed rounds : 23 
Total distance of all routes: 22345m
Total load of all routes: 44550


Warehouse :  6
Solver status:  1
Total needed rounds : 37 
Total distance of all routes: 59108m
Total load of all routes: 71303


Warehouse :  7
Solver status:  1
Total needed rounds : 18 
Total distance of all routes: 

In [48]:
df_route = pd.concat(tmp_list)

order = 121
tmp = df_route.query('order == '+str(order))

for prod in order_quantity_items[order]:
    assert np.isin(prod, tmp['product'])
    

df_route.shape

(23189, 5)

In [49]:
df_route = df_route.query('load != 0')
print("instruction for warehouse 1 : ", df_route.query("warehouse == "+str(1)).shape[0] )

instruction for warehouse 1 :  137


In [50]:
fig = make_subplots(rows=1, cols=1)

ref_color = ['red','blue',  'brown', 'yellow', 'cyan', 'orange', 'black', 'purple', 'pink', 'green']


for wh in warehouses:
    tmp = df_wh.iloc[wh]
    color = ref_color[wh]
    fig.add_trace(go.Scatter(x=[tmp['x']], y=[tmp['y']],
                             name='warehouse '+str(wh),
                             marker_size=15,
                             marker_color=color,
                             mode='markers', 
                            text='warehouse '+str(wh)
                            ), row=1, col=1)


    tmp_wh = df_wh.iloc[wh]

    orders = list(df_route.query("warehouse == "+str(wh)).order.unique())
    order_x = df_order.query("order in "+str(orders))['x']
    order_y = df_order.query("order in "+str(orders))['y']

    fig.add_trace(go.Scatter(x=order_x, y=order_y,
                            name='orders for warehouse '+str(wh),
                            marker_size=6,
                            marker_color=color,
                            mode='markers'), row=1, col=1)

    for r in df_route.query("warehouse == "+str(wh)).route.unique()[:60]: 
        route = df_route.query('warehouse == '+str(wh)+' and route == '+str(r))
        pre_node = (tmp_wh['x'], tmp_wh['y'])
        for i in range(route.shape[0]):
            if i != 0:
                order = route.iloc[i]['order']

                x = df_order['x'][order]
                y = df_order['y'][order]
                fig.add_trace(go.Scatter(x=(pre_node[0], x) , y=(pre_node[1], y), name="route "+str(r),marker_size=1, marker_color=color), row=1, col=1)
                pre_node = (x, y)

        fig.add_trace(go.Scatter(x=[pre_node[0], int(tmp_wh['x'])  ] , y=[pre_node[1], tmp_wh['y'] ], name="route "+str(r),marker_size=1, marker_color=color), row=1, col=1)


fig.show()

In [1]:
df_route['qty'] = [1]*df_route.shape[0]
df_route.groupby(['warehouse', 'route', 'product'])['qty'].sum().reset_index().query('qty > 1 and product != 0').shape()

NameError: name 'df_route' is not defined